In [21]:
import os
import yaml
import glob
import numpy as np
def extract_names_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Find the YAML front matter
    front_matter = []
    in_front_matter = False
    for line in lines:
        if line.strip() == '---':
            if not in_front_matter:
                in_front_matter = True
                continue
            else:
                break
        if in_front_matter:
            front_matter.append(line)
            # print(line)

    # Parse the YAML front matter
    front_matter_str = ''.join(front_matter)
    data = yaml.safe_load(front_matter_str)
    return data.get('title')  # Assuming 'name' is the key for names

def get_names_from_collections(collection_path):
    names = []
    print(collection_path)
    # Iterate over all Markdown files in the collection
    for file_path in glob.glob(os.path.join(collection_path, '*.md')):
        name = extract_names_from_file(file_path)
        if name:
            names.append(name)
    return names

In [25]:
collections = ['_pi','_postdocs','_grads','_undergrads']

names = np.concatenate([get_names_from_collections('../'+folder) for folder in collections ])
print(names)
names = [name.split(' ') for name in names]
names = ['^'+name[-1]+', '+name[0] for name in names]
sorted_names = sorted(names)
print(sorted_names)

../_pi
../_postdocs
../_grads
../_undergrads
['Marla Geha' 'Farnik Nikakhtar' 'Colin Burke' 'Erin Kado-Fong'
 'Will Cerny' 'Yasmeen Asali' 'Juan Guerra' 'name']
['^Asali, Yasmeen', '^Burke, Colin', '^Cerny, Will', '^Geha, Marla', '^Guerra, Juan', '^Kado-Fong, Erin', '^Nikakhtar, Farnik', '^name, name']


In [16]:
# ads part
import ads
ads.config.token = 'eZdDT6NdsJnTI4Rrxn7hDNcTdoaMTZ0uDTFPJBy3'

In [17]:
import ads
import os

def make_bib_md(author, publications_dir="_publications"):
    '''
    Fetch all papers from ADS and create Markdown files for each publication.

    Parameters:
    author (str): The author's name in the format "Last name, First name".
    publications_dir (str): The directory to store the Markdown files (default is "_publications").

    Examples:
    - Fetch papers by the first author:
    >> make_bib_md("^Last name, First name")

    - Fetch all papers by the author:
    >> make_bib_md("Last name, First name")

    Notes:
    - Make sure to install the 'ads' library before using this function.
    - You may need to configure your ADS credentials to use the library.
    '''
    try:
        papers = list(
            ads.SearchQuery(
                q=f'author:"{author}"',
                fl=[
                    "title",
                    "author",
                    "journal",
                    "year",
                    "citation_count",
                    "bibcode",
                ]
            )
        )
    except ads.exceptions.APIResponseError as e:
        print(f"Error fetching papers: {e}")
        return

    if not os.path.exists(publications_dir):
        os.makedirs(publications_dir)

    for p in papers:
        try:
            bibquery = ads.ExportQuery(p.bibcode)
            bibs = bibquery.execute()

            # Generate a Markdown file for each publication
            filename = os.path.join(publications_dir, f"{p.bibcode}.md")
            with open(filename, "w+") as md_file:
                md_file.write(f"---\n")
                md_file.write(f"title: \"{p.title}\"\n")
                md_file.write(f"author: \"{p.author}\"\n")
                md_file.write(f"journal: \"{p.journal}\"\n")
                md_file.write(f"year: \"{p.year}\"\n")
                md_file.write(f"---\n\n")
                md_file.write(f"{bibs}\n")

        except ads.exceptions.APIResponseError as e:
            print(f"Error exporting BibTeX for {p.title}: {e}")

    print(f"{len(papers)} Markdown files created in '{publications_dir}' directory.")


In [18]:
def make_bib(author,outfile="cv.bib"):
    '''
    fetch all files on ads and make a bibtext file from it

    Parameters
    ----------
    author : string
        
    Examples
    ----------    
    first author papers
    >> make_bib("^Last name, first_name")
    
    all papers
    >> make_bib("Last name, first_name") # note the removed

    Notes:
    
    '''
    papers = list(
            ads.SearchQuery(
                q=f'=author:\"{author}\"',
                fl=[
                    "citation_count",
                    "abbr",
                    "bibcode",
                ]
        )
        )
    
    with open(outfile,"w+")as out:
        for p in papers:
            bibquery = ads.ExportQuery(p.bibcode)
            bibs = bibquery.execute()
            out.write(bibs)

# if __name__ == '__main__':
#     make_bib("^Nikakhtar, Farnik",outfile='main.bib')

In [19]:
make_bib("^Nikakhtar, Farnik",outfile='test.bib')

In [26]:
import ads

def make_bib(authors, outfile="../_includes/most_recent_all.bib"):

    with open(outfile, "w+") as out:
        for author in authors:
            # Search for papers by the author, sorted by publication date (most recent first)
            papers = list(
                ads.SearchQuery(
                    q=f'author:"{author}"',
                    fl=[
                        "citation_count",
                        "abbr",
                        "bibcode",
                    ],
                    sort="date desc",  # Sort by publication date in descending order
                )
            )
            
            # Check if there are any papers for the author
            if papers:
                # Get the most recent paper (first paper in the list)
                most_recent_paper = papers[0]
                
                # Export the BibTeX entry for the most recent paper
                bibquery = ads.ExportQuery(most_recent_paper.bibcode)
                bibs = bibquery.execute()
                
                # Write the BibTeX entry to the output file
                out.write(bibs)

In [28]:
make_bib(sorted_names,outfile='most_recent_all.bib')